In [3]:
import sklearn
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.feature_extraction.text import TfidfVectorizer

In [4]:
scikit_version=sklearn.__version__
scikit_version

'0.23.1'

In [5]:
sentiment_data=pd.read_csv('C:/Users/shubham.chhetri/Desktop/MYPROJECT/Machine learning/Deploying ML Solutions/Sentimental analysis data.csv',header=None,names=['Label','Text'],sep='\t')
sentiment_data.sample(10)

,Label,Text
95,1,Oh oh oh and I loved The Da Vinci Code!
6613,0,Brokeback Mountain was boring.
1397,1,mission impossible 2 rocks!!....
5663,0,"Not because I hate Harry Potter, but because I..."
5977,0,brokeback mountain pretty much sucked..
723,1,The Da Vinci Code was absolutely AWESOME!
3174,1,I love Brokeback Mountain....
5865,0,These Harry Potter movies really suck.
347,1,The Da Vinci Code is awesome..
1286,1,i love kirsten / leah / kate escapades and mis...


In [6]:
sentiment_data.shape

(6918, 2)

In [7]:
x=sentiment_data['Text']
y=sentiment_data['Label']

In [8]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [9]:
x_train.shape,x_test.shape

((5534,), (1384,))

In [10]:
y_train.shape,y_test.shape

((5534,), (1384,))

In [11]:
tfidf_vect=TfidfVectorizer(max_features=15)
x_trans=tfidf_vect.fit_transform(x_train)

In [12]:
print(x_trans[0:3])

  (0, 1)	0.5150544336842106
  (0, 7)	0.4700955913574807
  (0, 3)	0.41373760314707125
  (0, 13)	0.41385399664049044
  (0, 4)	0.41385399664049044
  (1, 6)	0.5150387086400994
  (1, 9)	0.5150387086400994
  (1, 1)	0.5060786496235208
  (1, 7)	0.4619032989705869
  (2, 12)	0.43845581480852974
  (2, 10)	0.4967376886655494
  (2, 2)	0.4967376886655494
  (2, 7)	0.5605888295167891


In [13]:
x_trans.shape

(5534, 15)

In [15]:
classifier=LinearSVC(C=1.0,max_iter=1000,tol=1e-3)
linear_svc_model=classifier.fit(x_trans,y_train)
linear_svc_model

LinearSVC(tol=0.001)

In [16]:
x_test_trans=tfidf_vect.fit_transform(x_test)

In [17]:
x_test_trans.shape

(1384, 15)

In [18]:
y_pred=linear_svc_model.predict(x_test_trans)
y_pred

array([0, 0, 1, ..., 1, 1, 0], dtype=int64)

In [19]:
pred_results=pd.DataFrame({'y_test':y_test,'y_pred':y_pred})
pred_results.sample(5)

,y_test,y_pred
4250,0,0
4109,0,0
5467,0,0
357,1,1
6180,0,0


In [20]:
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_test,y_pred)
accuracy

0.9046242774566474

In [21]:
text_clf_param={}
text_clf_param['preprocessing']=tfidf_vect
text_clf_param['model']=linear_svc_model
text_clf_param['sklearn_version']=scikit_version
text_clf_param['accuracy']=accuracy

In [22]:
text_clf_param

{'preprocessing': TfidfVectorizer(max_features=15),
 'model': LinearSVC(tol=0.001),
 'sklearn_version': '0.23.1',
 'accuracy': 0.9046242774566474}

In [23]:
import joblib

In [24]:
filename='models/text_clf_checkpoint.joblib'

In [25]:
joblib.dump(text_clf_param,filename)

['models/text_clf_checkpoint.joblib']

In [26]:
clf_checkpoint=joblib.load(filename)

In [27]:
reloaded_vect=clf_checkpoint['preprocessing']
reloaded_vect

TfidfVectorizer(max_features=15)

In [28]:
clf_model=clf_checkpoint['model']
clf_model

LinearSVC(tol=0.001)

In [29]:
x_test_trans_new=reloaded_vect.fit_transform(x_test)

In [30]:
y_pred=clf_model.predict(x_test_trans_new)
y_pred

array([0, 0, 1, ..., 1, 1, 0], dtype=int64)

In [31]:
accuracy_score(y_test,y_pred)

0.9046242774566474

for Entire Pipeline

In [32]:
from sklearn.pipeline import Pipeline

In [33]:
clf_pipeline=Pipeline(steps=[('tfidf_vect',tfidf_vect),('classifier',classifier)])
pipeline_model=clf_pipeline.fit(x_train,y_train)

In [34]:
y_pred=pipeline_model.predict(x_test)

In [35]:
accuracy=accuracy_score(y_test,y_pred)
accuracy

0.9046242774566474

In [36]:
pipe_clf_param={}
pipe_clf_param['pipeline_clf']=pipeline_model
pipe_clf_param['sklear_version']=scikit_version
pipe_clf_param['accuracy']=accuracy

In [37]:
filename='models/pipe_clf_checkpoint.joblib'

In [38]:
joblib.dump(pipe_clf_param,filename)

['models/pipe_clf_checkpoint.joblib']

In [39]:
pipe_clf_checkpoint=joblib.load(filename)

In [40]:
reloaded_pipeline=pipe_clf_checkpoint['pipeline_clf']
reloaded_pipeline

Pipeline(steps=[('tfidf_vect', TfidfVectorizer(max_features=15)),
                ('classifier', LinearSVC(tol=0.001))])

In [41]:
y_pred=reloaded_pipeline.predict(x_test)

In [42]:
accuracy_score(y_test,y_pred)

0.9046242774566474